# Transformation
## 📚 강의 개요 (Overview)

이 강의에서는 RAG(Retrieval-Augmented Generation) 파이프라인에서 중요한 전처리 과정인 ‘Transformation’을 다룹니다. 여기서 ‘Transformation’은 단순한 스케일링이나 정규화 같은 전통적 의미의 데이터 변환이 아니라, 질의응답(Question-Answering) 및 생성(Generation)을 효과적으로 수행하기 위해 텍스트를 다양한 기준으로 나누는(Chunking) 과정을 의미합니다.

RAG 모델은 대용량 텍스트 데이터에서 필요한 정보를 검색(Retrieval)한 뒤, 검색된 결과를 입력으로 하여 응답(Generation)을 생성하는 구조입니다. 이때, 문서(또는 여러 형태의 텍스트)를 어떻게 분할(Chunking)하고, 어떤 임베딩을 사용해 의미를 추출하느냐가 모델 성능에 큰 영향을 미칩니다.

이 강의에서는 다음과 같은 Transformation(Chunking) 방법을 소개하며, 각각의 사용 용도와 장단점을 살펴봅니다:

## 목차: 
* [CharacterTextSplitter청킹](#charactertextsplitter-청킹)
* [RecursiveCharacterTextSplitter청킹](#recursivecharactertextsplitter-청킹)
* [코드 청킹](#코드-청킹)
* [마크다운 문서 청킹](#마크다운-문서-청킹)
* [시맨틱 청킹](#시맨틱-청킹)
* [오픈소스 임베딩 모델을 활용한 시맨틱 청킹](#오픈소스-임베딩-모델을-활용한-시맨틱-청킹)



In [12]:
# 환경변수 설정하기 (.env 파일을 사용하지 않을 경우 여기에 입력해주세요!)
import os

# 환경변수 설정
os.environ["API_KEY"] = "sk-..."

In [13]:
from dotenv import load_dotenv
load_dotenv(override = True)

True

In [14]:
# %pip install -qU langchain-text-splitters

### CharacterTextSplitter 청킹
텍스트를 일정 길이(캐릭터 수) 단위로 분할하는 가장 단순한 접근 방법입니다.

In [15]:
# PDF 파일 경로 설정
file_path = "data/arxiv_paper.pdf"

# LangChain의 PyPDFLoader를 이용해 PDF 파일을 로드
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 객체 생성
loader = PyPDFLoader(file_path)

# PDF의 각 페이지를 저장할 리스트
pages = []

# 비동기 방식으로 PDF 페이지를 로드 (async for 사용)
async for page in loader.alazy_load():
    pages.append(page)

In [16]:
# 텍스트 분할을 위한 CharacterTextSplitter 불러오기
from langchain_text_splitters import CharacterTextSplitter

# 텍스트 분할기 설정
text_splitter = CharacterTextSplitter(
    separator="\n\n",    # 문단 단위로 분할 (두 개의 개행 문자 기준)
    chunk_size=500,      # 하나의 청크(조각) 크기를 500자로 설정
    chunk_overlap=200,   # 청크 간 200자 겹치게 설정 (문맥 유지 목적)
    length_function=len, # 텍스트 길이를 측정하는 함수 (len 사용)
    is_separator_regex=False, # separator를 정규식이 아닌 단순 문자열로 처리
)

# PDF에서 로드한 데이터를 텍스트 청크로 분할
texts = text_splitter.split_documents(pages)

print(f"{texts[0].metadata}") # 첫 번째 청크의 메타데이터 
print(texts[0].page_content) # 첫 번째 청크의 내용 
print("-"*100)
print(f"{texts[1].metadata}") # 두 번째 청크의 메타데이터 
print(texts[1].page_content)# 두 번째 청크의 내용 

{'source': 'data/arxiv_paper.pdf', 'page': 0}
Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes. We
present the first work investigating embodied con-
fidence elicitation in open-ended multimodal en-
vironments. We introduce Elicitation Policies,
which structure confidence assessment across
inductive, deductive, and abductive reasoning,
along with Execution Policies, which enhance
confidence calibration through scenario reinter-
pretation, action sampling, and hypothetical rea-
soning. Evaluating agents in calibration and fail-
ure prediction t

In [17]:
print(f"""
      500 이상 문장 개수: {
          len(
              [i for i in texts if len(i.page_content) > 500]
              )
          }""")


      500 이상 문장 개수: 18


### RecursiveCharacterTextSplitter 청킹

`RecursiveCharacterTextSplitter`는 문장을 여러 계층의 구분자(예: 문단, 줄바꿈, 공백 등)를 기준으로 재귀적으로 나누는 역할을 합니다.

일반적인 `CharacterTextSplitter`보다 유연하게 텍스트를 나누는 데 사용됩니다.


In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# RecursiveCharacterTextSplitter를 사용하여 텍스트 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,       # 하나의 청크 크기를 500자로 설정
    chunk_overlap=200,    # 청크 간 200자 겹치게 설정 (문맥 유지 목적)
    length_function=len,  # 텍스트 길이를 측정하는 함수 (len 사용)
    is_separator_regex=False,  # separator가 정규식이 아님을 명시
)

# PDF에서 로드한 데이터를 텍스트 청크로 분할
texts = text_splitter.split_documents(pages)

print(f"{texts[0].metadata}") # 첫 번째 청크의 메타데이터 
print(texts[0].page_content) # 첫 번째 청크의 내용
print("-"*100)
print(f"{texts[1].metadata}") # 두 번째 청크의 메타데이터
print(texts[1].page_content) # 두 번째 청크의 내용

{'source': 'data/arxiv_paper.pdf', 'page': 0}
Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
----------------------------------------------------------------------------------------------------
{'source': 'data/arxiv_paper.pdf', 'page': 0}
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes. We
present the first work investigating embodied con-
fidence elicitation in open-ended multimodal en-
vironments. We introduc

In [19]:
print(f"""
      500 이상 문장 개수: {
          len(
              [i for i in texts if len(i.page_content) > 500]
              )
              }""")


      500 이상 문장 개수: 0


In [20]:
# 다양한 구분자를 활용한 RecursiveCharacterTextSplitter 설정
text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",   # 두 개의 개행 문자 (문단 분리)
        "\n",     # 단일 개행 문자 (줄바꿈)
        " ",      # 공백 (단어 단위 분리)
        ".",      # 마침표
        ",",      # 쉼표
        "\u200b",  # Zero-width space (보이지 않는 공백 문자)
        "\uff0c",  # Fullwidth comma (중국어, 일본어에서 사용)
        "\u3001",  # Ideographic comma (중국어, 일본어에서 사용)
        "\uff0e",  # Fullwidth full stop (중국어, 일본어에서 사용)
        "\u3002",  # Ideographic full stop (중국어, 일본어에서 사용)
        "",       # 마지막 분할 기준 (기본적으로 아무 구분자가 없을 경우)
    ],
    chunk_size=500,       # 기존 설정 유지
    chunk_overlap=200,    # 기존 설정 유지
    length_function=len,  # 기존 설정 유지
)

### 코드 청킹

코드 블록이나 함수 단위로 텍스트를 나누어, 코드 문서 처리에 특화된 청킹 기법을 다룹니다.

In [21]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

# 지원되는 언어 목록 출력
print([e.value for e in Language])

['cpp', 'go', 'java', 'kotlin', 'js', 'ts', 'php', 'proto', 'python', 'rst', 'ruby', 'rust', 'scala', 'swift', 'markdown', 'latex', 'html', 'sol', 'csharp', 'cobol', 'c', 'lua', 'perl', 'haskell', 'elixir', 'powershell']


In [22]:
# Python 코드에 대한 기본적인 구분자 확인
RecursiveCharacterTextSplitter.get_separators_for_language(Language.PYTHON)

['\nclass ', '\ndef ', '\n\tdef ', '\n\n', '\n', ' ', '']

In [23]:
# 샘플 Python 코드
PYTHON_CODE = """
def hello_world():
    print("Hello, World!")

# Call the function
hello_world()
"""

# Python 언어에 최적화된 text splitter 생성
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,  # 사용할 언어를 Python으로 설정
    chunk_size=50,             # 청크 크기를 50자로 설정
    chunk_overlap=0            # 청크 간 오버랩 없음
)
# Python 코드 문서를 분할하여 생성
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs

[Document(metadata={}, page_content='def hello_world():\n    print("Hello, World!")'),
 Document(metadata={}, page_content='# Call the function\nhello_world()')]

### 마크다운 문서 청킹

Markdown 헤더나 문서 구조를 기준으로 텍스트를 분할하여, 문서 구조를 활용하는 방법을 살펴봅니다.

In [24]:
markdown_text = """
# 🦜️🔗 LangChain

⚡ Building applications with LLMs through composability ⚡

## What is LangChain?

# Hopefully this code block isn't split
LangChain is a framework for...

As an open-source project in a rapidly developing field, we are extremely open to contributions.
"""
# LangChain의 RecursiveCharacterTextSplitter를 사용하여 Markdown 텍스트 분할
md_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN,  # Markdown 언어 지정
    chunk_size=60,  # 청크 크기를 60자로 설정
    chunk_overlap=0  # 청크 간 오버랩 없음
)

# Markdown 문서를 청크로 나누기
md_docs = md_splitter.create_documents([markdown_text])
md_docs

[Document(metadata={}, page_content='# 🦜️🔗 LangChain'),
 Document(metadata={}, page_content='⚡ Building applications with LLMs through composability ⚡'),
 Document(metadata={}, page_content='## What is LangChain?'),
 Document(metadata={}, page_content="# Hopefully this code block isn't split"),
 Document(metadata={}, page_content='LangChain is a framework for...'),
 Document(metadata={}, page_content='As an open-source project in a rapidly developing field, we'),
 Document(metadata={}, page_content='are extremely open to contributions.')]

In [25]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

# Markdown 형식의 샘플 텍스트
markdown_document = "# Foo\n\n    ## Bar\n\nHi this is Jim\n\nHi this is Joe\n\n ### Boo \n\n Hi this is Lance \n\n ## Baz\n\n Hi this is Molly"

# 헤더를 기준으로 Markdown을 분할하기 위한 규칙 설정
headers_to_split_on = [
    ("#", "Header 1"),   # `#`은 Header 1로 분류
    ("##", "Header 2"),  # `##`는 Header 2로 분류
    ("###", "Header 3"), # `###`는 Header 3로 분류
]

# 헤더 기반으로 Markdown을 분할하는 Splitter 생성
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)

# 설정한 헤더를 기준으로 Markdown을 분할
md_header_splits = markdown_splitter.split_text(markdown_document)
md_header_splits

[Document(metadata={'Header 1': 'Foo', 'Header 2': 'Bar'}, page_content='Hi this is Jim  \nHi this is Joe'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Bar', 'Header 3': 'Boo'}, page_content='Hi this is Lance'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Baz'}, page_content='Hi this is Molly')]

### 시맨틱 청킹

텍스트의 의미(Semantic) 기반으로 청크를 생성해, 문맥적으로 연관된 내용을 하나의 청크로 묶는 접근입니다.

In [26]:
# %pip install --q langchain_experimental

In [27]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

file_path = (
    "data/arxiv_paper.pdf"
)

from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 객체 생성
loader = PyPDFLoader(file_path)

# PDF의 각 페이지를 저장할 리스트
pages = []

# 비동기 방식으로 PDF의 각 페이지를 로드 (async for 사용)
async for page in loader.alazy_load():
    pages.append(page)

# SemanticChunker를 사용하여 의미 기반으로 텍스트를 분할
text_splitter = SemanticChunker(OpenAIEmbeddings())

# 문서를 의미적 청킹(Semantic Chunking) 수행
docs = text_splitter.split_documents(pages)

# 첫 번째 청크의 내용 출력
print(docs[0].page_content)


Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes. We
present the first work investigating embodied con-
fidence elicitation in open-ended multimodal en-
vironments. We introduce Elicitation Policies,
which structure confidence assessment across
inductive, deductive, and abductive reasoning,
along with Execution Policies, which enhance
confidence calibration through scenario reinter-
pretation, action sampling, and hypothetical rea-
soning. Evaluating agents in calibration and fail-
ure prediction tasks within the Minecraft envi-
ronment, we sh

In [28]:
print(f"총 {len(docs)}개 만큼의 문서로 청킹되었습니다.")
print([len(i.page_content) for i in docs])

# 각 청크의 메타데이터 및 내용 출력
for i in docs:
    print(i.metadata)       # 문서의 메타데이터 출력 (예: 페이지 번호 등)
    print(i.page_content)   # 분할된 청크의 내용 출력
    print("-" * 100)        # 구분선 출력

총 60개 만큼의 문서로 청킹되었습니다.
[1315, 786, 2014, 3140, 975, 1997, 351, 1086, 3077, 2863, 2445, 77, 1742, 3402, 495, 1021, 2477, 1700, 1395, 1979, 1610, 557, 3496, 591, 1561, 949, 433, 1544, 367, 130, 3492, 57, 235, 302, 499, 209, 204, 151, 258, 2165, 1594, 2424, 176, 463, 2, 1425, 1108, 2219, 2466, 335, 2792, 1828, 1623, 901, 120, 2044, 284, 1679, 613, 2]
{'source': 'data/arxiv_paper.pdf', 'page': 0}
Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes. We
present the first work investigating embodied con-
fidence elicitation in open-ended multimodal en-


### 오픈소스 임베딩 모델을 활용한 시맨틱 청킹  
오픈소스 임베딩 모델을 활용하여 텍스트의 의미를 벡터로 변환하고, 이를 기반으로 효과적인 청킹을 수행하는 방법을 실습합니다.  

**bge-m3**는 BGE(Bilingual General Embeddings) 시리즈 중 하나로, **텍스트 임베딩(embedding)**을 생성하는 강력한 모델입니다. 주로 정보 검색(Retrieval), 문서 분류, 시맨틱 검색(Semantic Search) 등 다양한 자연어 처리(NLP) 작업에서 활용됩니다.

In [29]:
%pip install langchain-ollama


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#"bge-m3" 모델을 다운로드 및 설치 (설치가 안 되어 있다면 다운로드)
# !ollama pull bge-m3


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling daec91ffb5dd... 100% ▕████████████████▏ 1.2 GB                         
pulling a406579cd136... 100% ▕████████████████▏ 1.1 KB                         
pulling 0c4c9c2a325f... 100% ▕████████████████▏  337 B                         
verifying sha256 digest 
writing manifest 
success 


In [31]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_ollama import OllamaEmbeddings

# PDF 파일 경로 설정
file_path = "data/arxiv_paper.pdf"

# LangChain의 PyPDFLoader를 사용하여 PDF 로드
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 객체 생성
loader = PyPDFLoader(file_path)

# PDF의 각 페이지를 저장할 리스트
pages = []

# 비동기 방식으로 PDF의 각 페이지를 로드 
async for page in loader.alazy_load():
    pages.append(page)

# SemanticChunker를 사용하여 의미 기반으로 텍스트를 분할
# OllamaEmbeddings의 "bge-m3" 모델을 사용하여 임베딩 생성
text_splitter = SemanticChunker(OllamaEmbeddings(model="bge-m3"))

# 문서를 의미적 청킹(Semantic Chunking) 수행
docs = text_splitter.split_documents(pages)

# 첫 번째 청크의 내용 출력
print(docs[0].page_content)


Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes. We
present the first work investigating embodied con-
fidence elicitation in open-ended multimodal en-
vironments. We introduce Elicitation Policies,
which structure confidence assessment across
inductive, deductive, and abductive reasoning,
along with Execution Policies, which enhance
confidence calibration through scenario reinter-
pretation, action sampling, and hypothetical rea-
soning. Evaluating agents in calibration and fail-
ure prediction tasks within the Minecraft envi-
ronment, we sh

In [32]:
print(f"총 {len(docs)}개 만큼의 문서로 청킹되었습니다.")

# 각 청크의 길이(문자 개수) 출력
print([len(i.page_content) for i in docs])

# 각 청크의 메타데이터 및 내용 출력
for i in docs:
    print(i.metadata)       # 문서의 메타데이터 출력 (예: 페이지 번호 등)
    print(i.page_content)   # 분할된 청크의 내용 출력
    print("-" * 100)       

총 60개 만큼의 문서로 청킹되었습니다.
[1315, 786, 2014, 3140, 1083, 1889, 1438, 838, 2238, 2863, 925, 1597, 1742, 3402, 495, 1021, 2477, 1700, 1392, 1982, 1610, 498, 58, 4088, 2511, 433, 1544, 213, 153, 597, 473, 1074, 1476, 596, 499, 414, 547, 79, 229, 1420, 1892, 2424, 176, 463, 2, 1425, 1108, 2219, 2463, 338, 2792, 1828, 1623, 901, 120, 2047, 281, 1679, 233, 382]
{'source': 'data/arxiv_paper.pdf', 'page': 0}
Uncertainty in Action: Confidence Elicitation in Embodied Agents
Tianjiao Yu, Vedant Shah, Muntasir Wahed, Kiet A. Nguyen, Adheesh Juvekar
Tal August, Ismini Lourentzou
University of Illinois Urbana-Champaign
{ty41,vrshah4,mwahed2,kietan2,adheesh2,taugust,lourent2}@illinois.edu
https://plan-lab.github.io/ece
Abstract
Expressing confidence is challenging for embod-
ied agents navigating dynamic multimodal en-
vironments, where uncertainty arises from both
perception and decision-making processes. We
present the first work investigating embodied con-
fidence elicitation in open-ended multimodal 